### Batches

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import os
import pickle
import copy
from PIL import Image
from skimage import exposure

In [ ]:
def saveToPickle(data, file_name=None, folder_path=None):
    if not os.path.isdir(folder_path):
        #print("Create \"preprocessed-data\" folder")
        os.mkdir(folder_path)
    else:
        print("\"preprocessed-data\" folder already exist")

    file_name = folder_path + file_name
    if not os.path.exists(file_name):
        try:
            with open(file_name, 'wb') as f:
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
                print("Create", file_name)
        except Exception as e:
            print('Error: unable to save data to', file_name, 'because', e)
            
def loadPickle(file_name=None, folder_path=None):
    file = folder_path + file_name
    #print('Load')
    if os.path.exists(file):
        try:
            with open(file, 'rb') as f:
                return pickle.load(f)
                print("Open", file)
        except Exception as e:
            print('Error: unable to open data to', file, 'because', e)

In [ ]:
## Load
augmented_X_train = loadPickle(file_name='augmented_X_train.p', folder_path='./preprocessed-data/')
augmented_y_train = loadPickle(file_name='augmented_y_train.p', folder_path='./preprocessed-data/')

augmented_X_valid = loadPickle(file_name='augmented_X_valid.p', folder_path='./preprocessed-data/')
augmented_y_valid = loadPickle(file_name='augmented_y_valid.p', folder_path='./preprocessed-data/')

augmented_X_test = loadPickle(file_name='augmented_X_test.p', folder_path='./preprocessed-data/')
augmented_y_test = loadPickle(file_name='augmented_y_test.p', folder_path='./preprocessed-data/')

print('The shape of the loaded processed X train dataset:', augmented_X_train.shape)
print('The shape of the loaded processed y train dataset:', augmented_y_train.shape)

print('The shape of the loaded processed X valid dataset:', augmented_X_valid.shape)
print('The shape of the loaded processed y valid dataset:', augmented_y_valid.shape)

print('The shape of the loaded processed X test dataset:', augmented_X_test.shape)
print('The shape of the loaded processed y test dataset:', augmented_y_test.shape)

In [ ]:
plt.imshow(augmented_X_train[7806], cmap='gray')

In [ ]:
plt.imshow(augmented_X_valid[2306], cmap='gray')

In [ ]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    if batch_size > 0:
        assert len(features) == len(labels)

        output_batches = []
        sample_size = len(features)

        for start_i in range(0, sample_size, batch_size):
            end_i = start_i + batch_size
            batch = [features[start_i:end_i], labels[start_i:end_i]]
            output_batches.append(batch)
    else:
        assert len(features) == len(labels)
        
        output_batches = []
        sample_size = len(features)
        
        for start in range(sample_size):
            batch = [features[start], labels[start]]
            output_batches.append(batch)
            
    return output_batches

### Model Architecture

In [ ]:
from tensorflow.contrib.layers import flatten

def InceptionModule(x, inputDepth):
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # 1x1 convolution
    a_conv_W_1x1 = tf.Variable(tf.truncated_normal(shape=(1, 1, inputDepth, 6), mean = mu, stddev = sigma))
    a_conv_b_1x1 = tf.Variable(tf.zeros(6))
    a_conv_1x1   = tf.nn.conv2d(x, a_conv_W_1x1, strides=[1, 1, 1, 1], padding='SAME') + a_conv_b_1x1
    print('a_conv_1x1:', np.shape(a_conv_1x1))
    
    # 3x3 convolution after 1x1 convolution
    b_conv_W_3x3 = tf.Variable(tf.truncated_normal(shape=(3, 3, 6, 32), mean = mu, stddev = sigma))
    b_conv_b_3x3 = tf.Variable(tf.zeros(32))
    b_conv_3x3   = tf.nn.conv2d(a_conv_1x1, b_conv_W_3x3, strides=[1, 1, 1, 1], padding='SAME') + b_conv_b_3x3
    print('b_conv_3x3:', np.shape(b_conv_3x3))
    
    # 5x5 convolution after 1x1 convolution
    c_conv_W_5x5 = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 43), mean = mu, stddev = sigma))
    c_conv_b_5x5 = tf.Variable(tf.zeros(43))
    c_conv_5x5   = tf.nn.conv2d(a_conv_1x1, c_conv_W_5x5, strides=[1, 1, 1, 1], padding='SAME') + c_conv_b_5x5
    print('c_conv_5x5:', np.shape(c_conv_5x5))
    
    # Pooling
    d_pooling = tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')
    print('d_Pooling:', np.shape(d_pooling))
    
    # 1x1 convolution after Pooling
    d_conv_W_1x1 = tf.Variable(tf.truncated_normal(shape=(1, 1, inputDepth, 6), mean = mu, stddev = sigma))
    d_conv_b_1x1 = tf.Variable(tf.zeros(6))
    d_conv_1x1   = tf.nn.conv2d(d_pooling, d_conv_W_1x1, strides=[1, 1, 1, 1], padding='SAME') + d_conv_b_1x1
    print('d_conv_1x1:', np.shape(d_conv_1x1))
    
    inceptionOutput = tf.concat([a_conv_1x1, b_conv_3x3, c_conv_5x5, d_conv_1x1], 3)
    print('Inception Output:', np.shape(inceptionOutput))
    return inceptionOutput

def Model(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    ## Layer 1: ##
    # Convolutional. Input = 32x32x1. Output = output 30x30x6
    L1_conv_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 6), mean = mu, stddev = sigma))
    L1_conv_b = tf.Variable(tf.zeros(6))
    L1_conv   = tf.nn.conv2d(x, L1_conv_W, strides=[1, 1, 1, 1], padding='VALID') + L1_conv_b
    print('L1-1:', np.shape(L1_conv))
    
    # Activation.
    L1_conv = tf.nn.relu(L1_conv)
    # conv = tf.nn.dropout(conv, 0.7)
    print('L1-2:', np.shape(L1_conv))
    
    ## Layer 2 ##
    # Convolutional. Input = 30x30x6. Output = 28x28x16.
    L2_conv_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 6, 16), mean = mu, stddev = sigma))
    L2_conv_b = tf.Variable(tf.zeros(16))
    L2_conv   = tf.nn.conv2d(L1_conv, L2_conv_W, strides=[1, 1, 1, 1], padding='VALID') + L2_conv_b
    print('L2-1:', np.shape(L2_conv))
    
    # Activation.
    L2_conv = tf.nn.relu(L2_conv)
    #conv1 = tf.nn.dropout(conv1, 0.7)
    print('L2-2:', np.shape(L2_conv))

    # Pooling. Input = 28x28x16. Output = 14x14x16.
    L2_conv = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    L2_depth = np.shape(L2_conv)[3].value
    print('L2-3:', np.shape(L2_conv))
    
    ## Layer 3: ##
    # Inception. Output = 10x10x64.
    L3_inception = InceptionModule(L2_conv, L2_depth)
    print('L3-1:', np.shape(L3_inception))
    
    # Activation.
    L3_inception = tf.nn.relu(L3_inception)
    L3_inception = tf.nn.dropout(L3_inception, 0.5)
    print('L3-2:', np.shape(L3_inception))
    
    ## Layer 4: ##
    # Convolutional. Input = 14x14x108. Output = 12x12x320.
    L4_conv_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 87, 230), mean = mu, stddev = sigma))
    L4_conv_b = tf.Variable(tf.zeros(230))
    L4_conv   = tf.nn.conv2d(L3_inception, L4_conv_W, strides=[1, 1, 1, 1], padding='VALID') + L4_conv_b
    print('L4-1:', np.shape(L4_conv))
    # Activation.
    L4_conv = tf.nn.relu(L4_conv)
    L4_conv = tf.nn.dropout(L4_conv, 0.7)

    # Pooling. Input = 12x12x320. Output = 3x3x320.
    L4_conv = tf.nn.max_pool(L4_conv, ksize=[1, 3, 3, 1], strides=[1, 3, 3, 1], padding='VALID')
    print('L4-2:', np.shape(L4_conv))

    # Flatten. Input = 3x3x320. Output = 2880.
    fc0   = flatten(L4_conv)
    print('fc0:', np.shape(fc0))
    
    ## Layer 5: ## 
    # Fully Connected. Input = 2880. Output = 1200.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(2070, 1700), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(1700))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    print('fc1:', np.shape(fc0))
    
    # Activation.
    fc1    = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, 0.5)

    ## Layer 5: ##
    # Fully Connected. Input = 1200. Output = 430.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(1700, 800), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(800))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Activation.
    fc2    = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, 0.5)
    print('fc2:', np.shape(fc0))
    
    # Layer 6: Fully Connected. Input = 430. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(800, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b

    return logits

### Train, Validate and Test the Model

In [ ]:
rate = 0.001
EPOCHS = 30
BATCH_SIZE = 32

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

## Features and Labels
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

In [ ]:
## Training Pipeline
#logits =  LeNet(x)
logits = Model(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
## Model Evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data, batch_size, sess):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()

    for batch_x, batch_y in tqdm(batches(batch_size, X_data, y_data)):
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
%%time
## Train the Model

with tf.Session() as session:
    session.run(tf. global_variables_initializer())
    num_examples = len(augmented_X_train)
    
    print("Training...")
    #print()
    for i in range(EPOCHS):
        shuffle_X, shuffle_y = shuffle(augmented_X_train, augmented_y_train)
        for batch_x, batch_y in tqdm(batches(BATCH_SIZE, shuffle_X, shuffle_y)):
            session.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        
        train_accuracy = evaluate(augmented_X_train, augmented_y_train, BATCH_SIZE, session)
        validation_accuracy = evaluate(augmented_X_valid, augmented_y_valid, BATCH_SIZE, session)
        test_accuracy = evaluate(augmented_X_test, augmented_y_test, BATCH_SIZE, session)
        
        print("EPOCH {0} ...".format(i+1))
        print('Train Accuracy = {:.3f}'.format(train_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Test Accuracy = {:.3f}".format(test_accuracy))
    
    #saver.save(session, './CNN3')
    print("Model Saved")